In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/testimages/Billie Eilish Wallpaper.jpeg
/kaggle/input/face-recognition-dataset/Dataset.csv
/kaggle/input/face-recognition-dataset/Faces/Faces/Courtney Cox_24.jpg
/kaggle/input/face-recognition-dataset/Faces/Faces/Henry Cavill_9.jpg
/kaggle/input/face-recognition-dataset/Faces/Faces/Charlize Theron_68.jpg
/kaggle/input/face-recognition-dataset/Faces/Faces/Courtney Cox_55.jpg
/kaggle/input/face-recognition-dataset/Faces/Faces/Billie Eilish_88.jpg
/kaggle/input/face-recognition-dataset/Faces/Faces/Robert Downey Jr_88.jpg
/kaggle/input/face-recognition-dataset/Faces/Faces/Lisa Kudrow_30.jpg
/kaggle/input/face-recognition-dataset/Faces/Faces/Priyanka Chopra_47.jpg
/kaggle/input/face-recognition-dataset/Faces/Faces/Alexandra Daddario_5.jpg
/kaggle/input/face-recognition-dataset/Faces/Faces/Hrithik Roshan_50.jpg
/kaggle/input/face-recognition-dataset/Faces/Faces/Claire Holt_29.jpg
/kaggle/input/face-recognition-dataset/Faces/Faces/Andy Samberg_13.jpg
/kaggle/input/face-recogniti

In [2]:
!pip install ultralytics
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install facenet-pytorch==2.5.2
!pip install transformers==4.28.1 safetensors==0.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1
import pickle
from google.colab import files
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

In [4]:
import shutil
from pathlib import Path
base_path = Path("/kaggle/working/dataset")
base_path.mkdir(parents=True, exist_ok=True)

(base_path / "images" / "train").mkdir(parents=True, exist_ok=True)
(base_path / "images" / "val").mkdir(parents=True, exist_ok=True)
(base_path / "labels" / "train").mkdir(parents=True, exist_ok=True)
(base_path / "labels" / "val").mkdir(parents=True, exist_ok=True)

In [5]:
import numpy as np
import random

mtcnn = MTCNN(keep_all=True)

def create_yolo_annotations_for_faces(image_folder, output_base_path):
    image_folder = Path(image_folder)
    images = list(image_folder.glob("**/*.jpg")) + list(image_folder.glob("**/*.png"))

    random.shuffle(images)
    split_idx = int(0.8 * len(images))
    train_images = images[:split_idx]
    val_images = images[split_idx:]

    def process_images(image_list, split_name):
        for img_path in image_list:
            img = cv2.imread(str(img_path))
            h, w = img.shape[:2]

            # Detect faces
            boxes, _ = mtcnn.detect(img)
            if boxes is None:
                continue

            # Save image
            dest_img_path = Path(output_base_path) / "images" / split_name / img_path.name
            shutil.copy(str(img_path), str(dest_img_path))

            # Write YOLO label
            label_path = Path(output_base_path) / "labels" / split_name / (img_path.stem + ".txt")
            with open(label_path, "w") as f:
                for box in boxes:
                    x1, y1, x2, y2 = box
                    # Convert to YOLO format
                    cx = ((x1 + x2) / 2) / w
                    cy = ((y1 + y2) / 2) / h
                    bw = (x2 - x1) / w
                    bh = (y2 - y1) / h
                    f.write(f"0 {cx} {cy} {bw} {bh}\n")

    process_images(train_images, "train")
    process_images(val_images, "val")

    print(f"processed {len(train_images)} training images and {len(val_images)} validation images")

original_images_path = "/kaggle/input/face-recognition-dataset/Original Images/Original Images"
create_yolo_annotations_for_faces(original_images_path, base_path)

processed 2049 training images and 513 validation images


In [6]:
yaml_path = base_path / "dataset.yaml"
with open(yaml_path, "w") as f:
    f.write(f"path: {base_path}\n")
    f.write(f"train: images/train\n")
    f.write(f"val: images/val\n")
    f.write("names:\n  0: face\n")

print(f"dataset.yaml created at {yaml_path}")

dataset.yaml created at /kaggle/working/dataset/dataset.yaml


In [7]:
from ultralytics import YOLO
model = YOLO("yolov8m.pt")  # Medium model for better accuracy

model.train(
    data=str(yaml_path),
    epochs=10,          # Increase training epochs
    imgsz=640,
    batch=16,
    name='face_detection',
    augment=True        # Enable augmentation
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=face_detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1159.9±710.3 MB/s, size: 95.2 KB)


train: Scanning /kaggle/working/dataset/labels/train... 1981 images, 0 backgrounds, 2 corrupt: 100%|██████████| 1981/1981 [00:01<00:00, 1244.99it/s]

...

                   all        501        587      0.988      0.844      0.868      0.742



10 epochs completed in 0.233 hours.
Optimizer stripped from runs/detect/face_detection/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/face_detection/weights/best.pt, 52.0MB

Validating runs/detect/face_detection/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:11<00:00,  1.44it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        501        587      0.983      0.847      0.867      0.724
Speed: 0.1ms preprocess, 18.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/face_detection


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78c6b8d869d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [8]:
best_model = YOLO('runs/detect/face_detection/weights/best.pt')
metrics = best_model.val()

print("📊 Validation Metrics:")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"Precision: {metrics.box.p[0]:.4f}")
print(f"Recall: {metrics.box.r[0]:.4f}")

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2167.8±850.1 MB/s, size: 1025.7 KB)


val: Scanning /kaggle/working/dataset/labels/val.cache... 501 images, 0 backgrounds, 0 corrupt: 100%|██████████| 501/501 [00:00<?, ?it/s]

val: /kaggle/working/dataset/images/val/Andy Samberg_68.jpg: corrupt JPEG restored and saved
val: /kaggle/working/dataset/images/val/Billie Eilish_22.jpg: corrupt JPEG restored and saved
val: /kaggle/working/dataset/images/val/Billie Eilish_80.jpg: corrupt JPEG restored and saved
val: /kaggle/working/dataset/images/val/Lisa Kudrow_8.jpg: corrupt JPEG restored and saved
val: /kaggle/working/dataset/images/val/Zac Efron_3.jpg: corrupt JPEG restored and saved



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:07<00:00,  4.47it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        501        587      0.987      0.843      0.875      0.745
Speed: 1.2ms preprocess, 8.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/val
📊 Validation Metrics:
mAP50-95: 0.7449
mAP50: 0.8752
Precision: 0.9869
Recall: 0.8433


In [9]:
model = YOLO('runs/detect/face_detection/weights/best.pt')
mtcnn = MTCNN(keep_all=True)
resnet = InceptionResnetV1(pretrained='vggface2').eval()

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [10]:
def generate_embeddings(dataset_path): # Removed ori_image from parameters
    ori_image = {} # Initialize ori_image inside the function

    # This loop processes each person's folder one by one
    try:
        for person_name in os.listdir(dataset_path):
            person_dir = os.path.join(dataset_path, person_name)
            if not os.path.isdir(person_dir):
                continue

            collected_embeddings = []
            print(f"\n📷 Processing {person_name}...")

            for img_file in os.listdir(person_dir):
                if img_file.lower().endswith((".jpg", ".jpeg", ".png")):
                    img_path = os.path.join(person_dir, img_file)
                    image_bgr = cv2.imread(img_path)
                    if image_bgr is None:
                        continue

                    detections = model.predict(image_bgr, verbose=False)
                    if detections and len(detections[0].boxes) > 0:
                        boxes = detections[0].boxes.xyxy.cpu().numpy()

                        for box in boxes:
                            x1, y1, x2, y2 = map(int, box[:4])
                            face_crop = image_bgr[y1:y2, x1:x2]

                            if face_crop.size == 0:
                                print(f"  ⚠️ Skipped an empty crop in file: {img_file}")
                                continue

                            try:
                                aligned_faces = mtcnn(
                                    cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
                                )
                            except Exception as e:
                                print(f"  ⚠️ MTCNN error on file {img_file}: {e}")
                                continue

                            if aligned_faces is None:
                                continue

                            if aligned_faces.ndim == 3:
                                aligned_faces = aligned_faces.unsqueeze(0)

                            for face in aligned_faces:
                                embedding_vec = (
                                    resnet(face.unsqueeze(0))
                                    .detach()
                                    .cpu()
                                    .numpy()
                                    .flatten()
                                )
                                collected_embeddings.append(embedding_vec)

            if collected_embeddings:
                ori_image[person_name] = collected_embeddings
                print(f"✅ {len(collected_embeddings)} embeddings saved for {person_name}.")
            else:
                print(f"⚠️ No embeddings generated for {person_name}.")

    except Exception as e:
        print(f"An error occurred during embedding generation: {e}")

    # Ensure ori_image is saved even if an error occurred during processing
    with open("ori_image.pkl", "wb") as emb_file:
        pickle.dump(ori_image, emb_file)
    print("\n💾 All embeddings saved successfully.")

In [11]:
embeddings_file = "ori_image.pkl"
if os.path.exists(embeddings_file):
    with open(embeddings_file, "rb") as f:
        ori_image = pickle.load(f)
    print("embeddings loaded.")
else:
    print("⚠️ Embeddings file not found. Generating new embeddings...")
    # This is the crucial step: CALL THE FUNCTION
    ori_image = generate_embeddings("/kaggle/input/face-recognition-dataset/Original Images/Original Images")

⚠️ Embeddings file not found. Generating new embeddings...

📷 Processing Alia Bhatt...
✅ 80 embeddings saved for Alia Bhatt.

📷 Processing Charlize Theron...
✅ 78 embeddings saved for Charlize Theron.

📷 Processing Zac Efron...
✅ 94 embeddings saved for Zac Efron.

📷 Processing Billie Eilish...
✅ 96 embeddings saved for Billie Eilish.

📷 Processing Jessica Alba...
✅ 108 embeddings saved for Jessica Alba.

📷 Processing Priyanka Chopra...
✅ 102 embeddings saved for Priyanka Chopra.

📷 Processing Natalie Portman...
✅ 105 embeddings saved for Natalie Portman.

📷 Processing Hrithik Roshan...
✅ 97 embeddings saved for Hrithik Roshan.

📷 Processing Tom Cruise...
✅ 58 embeddings saved for Tom Cruise.

📷 Processing Roger Federer...
✅ 75 embeddings saved for Roger Federer.

📷 Processing Henry Cavill...
✅ 106 embeddings saved for Henry Cavill.

📷 Processing Amitabh Bachchan...
✅ 71 embeddings saved for Amitabh Bachchan.

📷 Processing Brad Pitt...
✅ 121 embeddings saved for Brad Pitt.

📷 Processin

In [12]:
embeddings_file = "ori_image.pkl"
if os.path.exists(embeddings_file):
    with open(embeddings_file, "rb") as f:
        ori_image = pickle.load(f)
    print("embeddings loaded.")
else:
    print("⚠️ Embeddings file not found. Generating new embeddings...")
    # This is the crucial step: CALL THE FUNCTION
    ori_image = generate_embeddings("/kaggle/input/face-recognition-dataset/Original Images/Original Images")

embeddings loaded.


In [13]:
print("Path exists:", os.path.exists(original_images_path))
print("Items in folder:", os.listdir(original_images_path))


Path exists: True
Items in folder: ['Alia Bhatt', 'Charlize Theron', 'Zac Efron', 'Billie Eilish', 'Jessica Alba', 'Priyanka Chopra', 'Natalie Portman', 'Hrithik Roshan', 'Tom Cruise', 'Roger Federer', 'Henry Cavill', 'Amitabh Bachchan', 'Brad Pitt', 'Dwayne Johnson', 'Kashyap', 'Elizabeth Olsen', 'Camila Cabello', 'Vijay Deverakonda', 'Courtney Cox', 'Ellen Degeneres', 'Margot Robbie', 'Anushka Sharma', 'Lisa Kudrow', 'Marmik', 'Robert Downey Jr', 'Virat Kohli', 'Claire Holt', 'Andy Samberg', 'Alexandra Daddario', 'Akshay Kumar', 'Hugh Jackman']


In [14]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
yolo_model = YOLO('runs/detect/face_detection/weights/best.pt')

# Load the model for generating embeddings
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Load the known face embeddings
with open('ori_image.pkl', 'rb') as f:
    known_embeddings_dict = pickle.load(f)
print("✅ Models and database loaded.")


# --- 2. Create a Function to Find Matches ---
def find_match(new_embedding, threshold=1.0):
    """
    Compares a new face embedding to the known embeddings in the database.
    """
    best_match_name = "Unknown"
    min_distance = float('inf')

    # Compare the new embedding with all stored embeddings for each person
    for name, stored_embeddings_list in known_embeddings_dict.items():
        for stored_embedding in stored_embeddings_list:
            distance = np.linalg.norm(new_embedding - stored_embedding)
            if distance < min_distance:
                min_distance = distance
                # If the closest distance is within the threshold, we have a match
                if min_distance < threshold:
                    best_match_name = name
                else:
                    best_match_name = "Unknown"

    return best_match_name, min_distance


# --- 3. Main Loop for Real-Time Recognition ---
# This part is intended for a local machine with a webcam.
# To run in Colab, you would need to use a different method for video input.
try:
    cap = cv2.VideoCapture(0) # 0 is the default webcam
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        exit()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Use YOLO to detect faces
        detections = yolo_model.predict(frame, verbose=False)

        if detections and len(detections[0].boxes) > 0:
            boxes = detections[0].boxes.xyxy.cpu().numpy()
            for box in boxes:
                x1, y1, x2, y2 = map(int, box[:4])

                # Crop the face
                face_crop = frame[y1:y2, x1:x2]

                if face_crop.size > 0:
                    # Convert crop to tensor for ResNet
                    face_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
                    face_tensor = torch.from_numpy(face_rgb).permute(2, 0, 1).float().div(255).unsqueeze(0).to(device)

                    # Generate embedding
                    embedding = resnet(face_tensor).detach().cpu().numpy().flatten()

                    # Find the best match
                    name, distance = find_match(embedding)

                    # Draw rectangle and text
                    label = f"{name} ({distance:.2f})"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow('Face Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

except Exception as e:
    print(f"An error occurred during webcam processing: {e}")
    print("Note: cv2.imshow() does not work in standard Google Colab cells.")
    print("This part of the code is designed for a local Python environment.")

✅ Models and database loaded.
Error: Could not open webcam.
An error occurred during webcam processing: OpenCV(4.11.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'

Note: cv2.imshow() does not work in standard Google Colab cells.
This part of the code is designed for a local Python environment.


[ WARN:0@2461.280] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@2461.281] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
